In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy.stats import norm

# U.S. STOCK MARKET RETURN

## ANNUAL

In [ ]:
mkt_annual = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt_annual = mkt_annual['Mkt-RF'] + mkt_annual.RF
mkt_annual.index.name = "year"
mkt_annual.name = "market annual"
mkt_annual.index = mkt_annual.index.astype(str)
mkt_annual.tail()

## MONTHLY

In [ ]:
mkt_monthly = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[0]/100
mkt_monthly = mkt_monthly['Mkt-RF'] + mkt_monthly.RF
mkt_annual.index.name = "month"
mkt_monthly.name = "market monthly"
mkt_monthly.index = mkt_monthly.index.astype(str)
mkt_monthly.tail()

## DAILY

In [ ]:
mkt_daily = pdr('F-F_Research_Data_Factors_Daily','famafrench',start=1926)[0]/100
mkt_daily = mkt_daily['Mkt-RF'] + mkt_daily.RF
mkt_daily.index.name = "day"
mkt_daily.name = "market daily"
mkt_daily.index = mkt_daily.index.astype(str)
mkt_daily.tail()

# RETURNS FROM YAHOO FINANCE

In [ ]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]

## ANNUAL

In [ ]:
price_annual = price.resample("Y").last()
ret_annual = price_annual.pct_change().dropna()[:-1]
ret_annual.index = ret_annual.index.map(lambda x: x.year)
ret_annual.index.name = "year"
ret_annual.name = ticker + " annual"
ret_annual.tail()

## MONTHLY

In [ ]:
price_monthly = price.resample("M").last()
ret_monthly = price_monthly.pct_change().dropna()[:-1]
ret_monthly.index = ret_monthly.index.to_period("M").astype(str)
ret_monthly.index.name = "month"
ret_monthly.name = ticker + " monthly"
ret_monthly.tail()

## DAILY

In [ ]:
ret_daily = price.pct_change().dropna()
ret_daily.index = ret_daily.index.to_period("D").astype(str)
ret_daily.index.name = "day"
ret_daily.name = ticker + " daily"
ret_daily.tail()

# SELECT YAHOO RETURN OR MARKET

In [ ]:
"Market Return"

flag = input("input 0 for market or 1 for yahoo return")
data = "ret" if flag else "mkt"
name = ticker.upper() if flag else "Market"
annual = eval(data + "_annual")
monthly = eval(data + "_monthly")
daily = eval(data + "_daily")

# TABLE

In [ ]:
ser1 = annual.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
ser2 = monthly.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
ser3 = daily.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
table = pd.concat((ser1, ser2, ser3), axis=1)
table

# FIGURE 1

In [ ]:
trace1 = go.Box(
    y=daily, 
    text=daily.index.to_list(), 
    name=name + " Daily", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
trace2 = go.Box(
    y=monthly, 
    text=monthly.index.to_list(), 
    name=name + " Monthly", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
trace3 = go.Box(
    y=annual, 
    text=annual.index.to_list(), 
    name=name + " Annual", 
    hovertemplate="%{text}<br>%{y:.1%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace2, trace3]:
    fig.add_trace(trace)
fig.update_layout(
    yaxis_tickformat=".1%",
    template="plotly_white",
    showlegend=False
)
fig.show()

# FIGURE 2

In [ ]:
trace1 = ff.create_distplot(
    [daily], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = daily.mean()
s = daily.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Daily Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()


# FIGURE 3

In [ ]:
trace1 = ff.create_distplot(
    [monthly], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = monthly.mean()
s = monthly.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Monthly Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()


# FIGURE 4

In [ ]:
trace1 = ff.create_distplot(
    [annual], 
    group_labels=["actual"], 
    show_rug=False, 
    show_hist=False
)

m = annual.mean()
s = annual.std()
grid = np.linspace(m - 3 * s, m + 3 * s, 201)
trace2 = go.Scatter(
    x=grid, 
    y=norm.pdf(grid, loc=m, scale=s), 
    mode="lines", 
    name="normal"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    yaxis_tickvals=[],
    xaxis_tickformat=".0%",
    title=dict(
        text=name + " Annual Returns",
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ),
    template="plotly_white",
)
fig.show()

